In [1]:
import pymysql
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import nltk

In [2]:
# db = pymysql.connect(host='localhost',
#                      user='root',
#                      password='Czy026110',
#                      database='homework')
db = pymysql.connect(host='124.70.198.102',
                     user='root',
                     password='HaRdEsTnju@123',
                     database='sentistrength')
cursor = db.cursor()

In [3]:
project = 'kafka'
version = 'docs'
select_content_sql = "SELECT content FROM data WHERE version_number = \'" + version + "\'" + " and project_name = \'" + project + "\'"
cursor.execute(select_content_sql)
content_list = cursor.fetchall()
content_results = [row[0] for row in content_list]

In [4]:
combined_content = [' '.join(content_results)]
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(combined_content)
feature_names = vectorizer.get_feature_names_out()

In [5]:
tfidf_array = tfidf.toarray()
top_n_idx = np.argsort(tfidf_array[0])[-100:]
top_n_values = [tfidf_array[0][i] for i in top_n_idx]
top_n_words = [feature_names[i] for i in top_n_idx]

In [6]:
word_dict = dict(zip(top_n_words, top_n_values))

In [7]:
nltk.download('averaged_perceptron_tagger')
nltk.download('nps_chat')
nltk_data = nltk.corpus.nps_chat.tagged_words()
nouns = [word.lower() for (word, tag) in nltk_data if tag.startswith('N')]
res_list = []
for key in list(word_dict.keys()):
    if key not in nouns:
        # 删除键及其对应的值
        del word_dict[key]
    else:
        word_dict[key] = int(word_dict[key] * 100 // 1)
        temp = {'name':key, 'value':word_dict[key]}
        res_list.append(temp)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/seedoilz/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package nps_chat to
[nltk_data]     /Users/seedoilz/nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!


In [8]:
print(res_list)

[{'name': 'system', 'value': 2}, {'name': 'guide', 'value': 2}, {'name': 'title', 'value': 2}, {'name': 'call', 'value': 2}, {'name': 'up', 'value': 2}, {'name': 'comment', 'value': 2}, {'name': 'site', 'value': 4}, {'name': 'as', 'value': 4}, {'name': 'docs', 'value': 4}, {'name': 'not', 'value': 5}, {'name': 'change', 'value': 5}, {'name': 'that', 'value': 6}, {'name': 'com', 'value': 6}, {'name': 'thanks', 'value': 6}, {'name': 'notes', 'value': 7}, {'name': 'test', 'value': 9}, {'name': 'it', 'value': 10}, {'name': 'in', 'value': 11}, {'name': 'of', 'value': 11}, {'name': 'pr', 'value': 15}, {'name': 'for', 'value': 25}, {'name': 'and', 'value': 33}, {'name': 'the', 'value': 43}]
